# Evaluated Exercise - Part 1: Databases

Please upload an html-version of your final version into the drop-zone in Moodle. If you have any issues with it, send the final version to guido.moeser@gmail.com.   
Release Date: 2025-10-07

## Final Submission Instructions

1. Complete all sections in the notebook.
2. Add explanations of all parts. **Explanations are the most important part for the grading.**
3. Comment on which configuration you found best and why. **Comments are the second most important part for the grading.**
4. Export your Jupyter Notebook to HTML and send the HTML-version.


# Topic: Building a simple Retrieval-Augmented Retrieval System with SQLite (in-memory)

## 1 – Import the required packages

**Required packages etc:**
- sqlite3
- pandas
- numpy
- from sklearn.metrics.pairwise we need cosine_similarity
- from the SentenceTransformer package we need the SentenceTransformer class


In [1]:
# Load the minimal set of packages we will need
# Uncomment below lines to install if using pixi package manager
#%pixi add pandas, numpy
import numpy as np
import pandas as pd
import sqlite3
from sentence_transformers import SentenceTransformer

**Explanation:**
- We will only use the core Python SQLite library and three data-science packages: pandas, numpy, and sentence-transformers.
- No extra dependencies are needed.

## 2 – Provide your own texts

Please insert 10 texts of length > 200 words (news articles, wikipedia article (parts of it), product descriptions, product reviews etc.).

In [ ]:
# Replace these sample texts with 10 texts of your own choice, length > 200 word.
texts = [
    "Text 1 – replace me with your own text.",
    "Text 2 – replace me with your own text.",
    # ...
    "Text 10 – replace me with your own text."
]

## 3 – Load the SentenceTransformer model

**Tasks:**
- Load the `all-MiniLM-L6-v2` using `SentenceTransformer()` and apply it to a sentence of your choice to show the embeddings.
- Explain what the embeddings are on a short sentence.

In [ ]:
# Load a lightweight embedding model from Huggingface
model = SentenceTransformer("all-MiniLM-L6-v2")

**Explanation:** ...


**Fidel's answer:** <br>
Embeddings are numerical column vector representation of words, sentences, paragraphs, etc. In order to achieve vector representation of the words, sentences, or paragraphs; these have to be split into **tokens** and each token is converted to a numerical value and these numeric values are grouped togheter on the size of the **context window**; the context window is choosen based on many approaches(similarity, context, word reelevance, etc.). Ultimately, these numbers are put in a single column vector which is the vector that embeds a representation of the word, sentences, or paragraphs.

## 4 – Create and fill an in-memory SQLite database

- Create an in-memory database with SQLite
- Create a table with the fields
  - id,
  - title,
  - text,
  - embedding,
  - import_time
- Metadata: Just add the time you loaded the data into the database
- Load the data into the database: `INSERT INTO documents (title, text, embedding, import_time) VALUES (?, ?, ?, ?)`
- Use pandas to run a SQL-request against the table to show that everything works fine

**Task:** Add the necessary SQL query. All other parts are already there.

In [ ]:
# Create an in-memory database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create a simple table (embedding stored as TEXT)
cursor.execute("""







""")

# Insert documents with metadata
import datetime

for i, t in enumerate(texts):
    title = f"Doc_{i+1}"
    emb = model.encode([t])[0]
    emb_str = ",".join(map(str, emb))   # convert vector to comma-separated string
    cursor.execute(
        "INSERT INTO documents (title, text, embedding, import_time) VALUES (?, ?, ?, ?)",
        (title, t, emb_str, str(datetime.datetime.now()))
    )

conn.commit()

# Test query using pandas
pd.read_sql("SELECT id, title, import_time FROM documents", conn)

## 5 – Define and test three similarity metrics

- **Task:** Please build to more functions. The functions should return a similarity score between two vectors.
- Will be used later to compare which metric retrieves the most relevant texts.

Here is one function:

```
# Metric 1: cosine similarity (from scikit-learn)
def cosine_sim(a, b):
    return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0]
```



In [ ]:
# Metric 1: cosine similarity (from scikit-learn)
def cosine_sim(a, b):
    return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0]

# Metric 2:
def


# Metric 3:
def


## 6 – Build a simple Retriever

A function that encodes a query, computes the similarity between the query and each document embedding, and returns the top 3 most similar texts by default.  
  
**Task: Code is complete, please explain what happens here**

In [ ]:
def retrieve(query, metric_function, top_k=3):
    # Encode the query
    q_vec = model.encode([query])[0]

    # Load all document embeddings
    cursor.execute("SELECT id, title, text, embedding FROM documents")
    docs = cursor.fetchall()

    results = []
    for doc_id, title, text, emb_str in docs:
        emb = np.fromstring(emb_str, sep=",")     # convert text back to numeric vector
        score = metric_function(q_vec, emb)
        results.append((title, text, score))

    # Sort by similarity and return top_k results
    results = sorted(results, key=lambda x: x[2], reverse=True)[:top_k]
    return pd.DataFrame(results, columns=["Title", "Text", "Score"])

**Explanation**
...

## 7 – Run a query and inspect the results

- Run the retriever with one of the metrics (cosine_sim, dot_product_sim, or inv_euclidean_sim).
- Check how the ranking of results changes across metrics.

In [ ]:
query = "Enter your own test question here"
retrieve(query, cosine_sim)

# Fine-Tuning Phase

## 8 – Reload database with different chunk sizes and overlaps

- Reload your in-memory database with various chunk_size and overlap settings (e.g. 30/10, 60/20).
- For each configuration, insert each chunk as a new row in the documents table and repeat the insertion logic from Section 4 (using comma-separated embeddings).

**Task:** Explain what happens here and test the functions.


In [ ]:
def chunk_text(text, chunk_size=50, overlap=10):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i+chunk_size])
        if chunk.strip():
            chunks.append(chunk)
    return chunks

**Explanations:** ...

In [ ]:
# Test the function

## 9 – Query the new database with different similarity metrics

Compare retrieval quality under the different metrics developed above. A record which combination of metric, chunk size, and overlap yields the most meaningful matches will be printed out.

**Task:** Replace the similarity-functions here with the functions you developed above:

```
for metric in [cosine_sim, <your sim function>, <your sim function>]:
    print(f"Results using {metric.__name__}:")
    display(retrieve(query, metric))
```

In [ ]:
for metric in [cosine_sim, dot_product_sim, inv_euclidean_sim]:
    print(f"Results using {metric.__name__}:")
    display(retrieve(query, metric))

## 10 – Use the systematic evaluation module

- This module systematically tests different configurations (chunk size, overlap, metric) and records which text was ranked highest.
- Visualize or summarize the outcomes to decide which configuration works best.

**Task:** Explain what happens here and run the experiment with different settings for
- chunksize
- overlap
- similarity functions

(Modify the experiment if you want, but not necessary or required).

*Please note: Replace the similarity functions with your similarity functions, otherwise it will throw an error.*

In [ ]:
def evaluate_configs(query, chunk_sizes, overlaps, metrics):
    results = []

    for cs in chunk_sizes:
        for ov in overlaps:
            # Rebuild an in-memory DB for each configuration
            conn = sqlite3.connect(":memory:")
            cursor = conn.cursor()
            cursor.execute("CREATE TABLE docs (id INTEGER PRIMARY KEY, text TEXT, embedding TEXT)")

            # Insert chunks with textual embeddings
            for t in texts:
                for ch in chunk_text(t, cs, ov):
                    emb = model.encode([ch])[0]
                    emb_str = ",".join(map(str, emb))
                    cursor.execute("INSERT INTO docs (text, embedding) VALUES (?, ?)", (ch, emb_str))
            conn.commit()

            # Encode the query once
            q_vec = model.encode([query])[0]

            # Evaluate all metrics
            for metric in metrics:
                cursor.execute("SELECT id, text, embedding FROM docs")
                docs = cursor.fetchall()
                scores = []

                for _, text, emb_str in docs:
                    emb = np.fromstring(emb_str, sep=",")
                    score = metric(q_vec, emb)
                    scores.append((text, score))

                # Sort by similarity and take the best one
                top_text, top_score = sorted(scores, key=lambda x: x[1], reverse=True)[0]

                results.append((cs, ov, metric.__name__, top_text, top_score))

    # Return results as DataFrame
    return pd.DataFrame(results, columns=["ChunkSize", "Overlap", "Metric", "TopResult", "SimilarityScore"])

**Explanations:** ...

**Example usage**

In [ ]:
# Define your test query
query = "Summarize the main idea of my texts"

# Define parameter grid
chunk_sizes = [30, 50, 70]
overlaps = [5, 10, 15]
metrics = [cosine_sim, dot_product_sim, inv_euclidean_sim]

# Run evaluation
results_df = evaluate_configs(query, chunk_sizes, overlaps, metrics)

# Display full table
display(results_df)

# Optional: Find the highest similarity overall
best_config = results_df.sort_values("SimilarityScore", ascending=False).head(1)
display(best_config)


**Interpretation of Results** ...